In [6]:
from torch.utils.data import Dataset, DataLoader
from dataset import AdverseWeatherDataset
import matplotlib.pyplot as plt
from torchvision import transforms
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from model import Model
import time
import datetime
from torch.autograd import Variable  
import kornia.metrics as metrics
import cv2
import os

In [7]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
TEST_DIR = r'./data/test'
BATCH_SIZE = 1
NUM_WORKERS = 4

In [8]:
test_set = AdverseWeatherDataset(root_dir=TEST_DIR, is_train=False)

test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)

In [9]:
# constrcut model
model = Model().cuda()
model = nn.DataParallel(model)

# load the trained model
model.load_state_dict(torch.load('checkpoints/0.pth'))
model.eval()

DataParallel(
  (module): Model(
    (encoder): Encoder(
      (original_model): MobileNetV2(
        (features): Sequential(
          (0): ConvNormActivation(
            (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU6(inplace=True)
          )
          (1): InvertedResidual(
            (conv): Sequential(
              (0): ConvNormActivation(
                (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
                (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                (2): ReLU6(inplace=True)
              )
              (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            )
          )
          (2): Invert

In [10]:
# Save predicted depth maps to folder
if not os.path.isdir: os.mkdir('result/generated_img')

for i,sample_batched1  in enumerate (test_loader):
    image1 = torch.autograd.Variable(sample_batched1['image'].cuda())

    with torch.no_grad():
        pred_depth=model(image1 ) # predicted depth map tensor
    
    x=np.array(pred_depth.cpu().detach()) # gets a numpy array from the tensor

    #rescale the depth map to original image size
    img=x.reshape(240,320)
    scale_percent = 200 # percent of original size
    width = int(img.shape[1] * scale_percent / 100)
    height = int(img.shape[0] * scale_percent / 100)
    dim = (width, height)
    resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    
    # save the rescale final depth map to disk
    plt.imsave('result/generated_img/%d_depth.jpg' %i, resized, cmap='inferno') 
    
    s_img=sample_batched1['image'].detach().cpu().numpy().reshape(3,480,640).transpose(1,2,0)
    plt.imsave('result/generated_img/%d_image.jpg' %i, s_img) 